In [16]:
import sys
sys.path.insert(0, '../')
#from models.model_mrcnn import _default_mrcnn_config, build_default
import torchvision
import torch
from models_pytorch_lightning.generalized_mask_rcnn_pl import LitMaskRCNN
import torchvision.ops.boxes as bops
import torchvision
from torchvision import transforms
import torch
from PIL import Image
import cv2
import numpy as np
import os
import pandas as pd
import os
import pyvips as Vips
from Reinhard import Reinhard
from openslide import OpenSlide
from PIL import Image, ImageDraw, ImageFont
from glob import glob
import random
from infer_utils import get_outputs_nms, prepare_input
import numpy as np
import cv2

In [2]:
model_name = "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/runpod_mrcnn_models/yp2mf3i8_epoch=108-step=872.ckpt"
model = LitMaskRCNN.load_from_checkpoint(model_name)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.eval().to(device)
class_names = ['Cored', 'Diffuse', 'Coarse-Grained', 'CAA']

/home/mahirwar/miniconda3/envs/kfold_amy_plaque1/lib/python3.9/site-packages/lightning/fabric/utilities/cloud_io.py:55: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
/home/mahirwar/miniconda

In [3]:
path = "/home/mahirwar/Desktop/Monika/npsad_data/vivek/Datasets/External_Dataset-Others/data/custom/labels"
label_list = os.listdir(path)
img_folder =  "/home/mahirwar/Desktop/Monika/npsad_data/vivek/Datasets/External_Dataset-Others/data/custom/images"
img_names =  [ os.path.join(img_folder,label_list[i].replace(".txt",".jpg")) for i in range(len(label_list))]

In [7]:
# Function to initialize normalization based on the reference image
def initialize_normalizer(ref_slide_path):
    print("Initializing normalization with reference slide:", ref_slide_path)
    ref_image = Vips.Image.new_from_file(ref_slide_path)
    normalizer = Reinhard()
    normalizer.fit(ref_image)
    return normalizer


# Function to normalize a single slide
def normalize_slide(target_image, normalizer, filepath, idx, output_dir):
    #print("Normalizing slide:", target_slide_path)
    # Load the target slide
    #target_image = Vips.Image.new_from_file(target_slide_path)
    target_image = Vips.Image.new_from_memory(target_image.data, target_image.shape[1], target_image.shape[0], 3, "uchar")
    # Perform normalization
    normalized_image = normalizer.transform(target_image)
    # Generate the output file path
    #file_name = os.path.basename(target_slide_path).replace(".mrxs", ".tif")
    output_file_path = os.path.join(output_dir, idx+"_" +filepath.split("/")[-1])
    # Save the normalized image
    normalized_image.write_to_file(output_file_path)
    return normalized_image

In [8]:
def convert_bbox(bbox):
    """
    Convert bounding box from [center_x, center_y, width, height] 
    to [x_min, y_min, x_max, y_max].

    Parameters:
        center_x (float): Center x-coordinate (normalized, 0-1).
        center_y (float): Center y-coordinate (normalized, 0-1).
        w (float): Width of the bounding box (normalized, 0-1).
        h (float): Height of the bounding box (normalized, 0-1).
    
    Returns:
        list: [x_min, y_min, x_max, y_max] (normalized)
    """
    center_x, center_y, w, h = bbox
    x_min = center_x - w / 2
    y_min = center_y - h / 2
    x_max = center_x + w / 2
    y_max = center_y + h / 2

    return [x_min, y_min, x_max, y_max]

In [9]:
def zoom_and_crop(image, bboxes, labels, zoom_factor=2, crop_size=(1024, 1024)):
    """
    Zoom the image and crop it into patches of specified size while adjusting bounding boxes.
    
    Parameters:
        image (numpy array): Original image of size (1536, 1536, 3)
        bboxes (list): List of normalized bounding boxes [[x_min, y_min, x_max, y_max], ...]
        zoom_factor (int): Factor to zoom the image.
        crop_size (tuple): Size of the crop (height, width)
        
    Returns:
        crops (list): List of cropped images.
        cropped_bboxes (list): List of bounding boxes adjusted for each crop.
    """
    # Step 1: Zoom the image
    original_h, original_w = image.shape[:2]
    zoomed_h, zoomed_w = int(original_h * zoom_factor), int(original_w * zoom_factor)
    zoomed_image = cv2.resize(image, (zoomed_w, zoomed_h), interpolation=cv2.INTER_LINEAR)
    # Step 2: Scale bounding boxes to match the zoomed image
    scaled_bboxes = []
    for bbox in bboxes:
        x_min, y_min, x_max, y_max = bbox
        x_min = int(x_min * zoomed_w)
        y_min = int(y_min * zoomed_h)
        x_max = int(x_max * zoomed_w)
        y_max = int(y_max * zoomed_h)
        scaled_bboxes.append([x_min, y_min, x_max, y_max])

    # Step 3: Generate crops and adjust bounding boxes
    crops = []
    cropped_bboxes = []
    cropped_labels = []

    crop_h, crop_w = crop_size

    for y in range(0, zoomed_h, crop_h):
        for x in range(0, zoomed_w, crop_w):
            # Define the crop region
            x1, y1 = x, y
            x2, y2 = min(x + crop_w, zoomed_w), min(y + crop_h, zoomed_h)
            crop = zoomed_image[y1:y2, x1:x2]

            # Adjust bounding boxes for this crop
            bboxes_in_crop = []
            labels_in_crop = []
            for bbox,label in zip(scaled_bboxes,labels):
                bx_min, by_min, bx_max, by_max = bbox

                # Check if the bounding box intersects with the crop
                if bx_max > x1 and bx_min < x2 and by_max > y1 and by_min < y2:
                    # Adjust the bounding box to the crop's coordinate system
                    new_x_min = max(bx_min - x1, 0)
                    new_y_min = max(by_min - y1, 0)
                    new_x_max = min(bx_max - x1, crop_w)
                    new_y_max = min(by_max - y1, crop_h)
                    bboxes_in_crop.append([new_x_min, new_y_min, new_x_max, new_y_max])
                    labels_in_crop.append(label)
            # Save the crop and corresponding bounding boxes
            crops.append(crop)
            cropped_bboxes.append(bboxes_in_crop)
            cropped_labels.append(labels_in_crop)

    return crops, cropped_bboxes,cropped_labels

In [10]:
#imgs = glob(os.path.join("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/normalized_data/images","*"))
#lbls = glob(os.path.join("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/normalized_data/bboxes","*"))


In [11]:
path = "/home/mahirwar/Desktop/Monika/npsad_data/vivek/Datasets/External_Dataset-Others/data/custom/labels"
label_list = os.listdir(path)

img_folder =  "/home/mahirwar/Desktop/Monika/npsad_data/vivek/Datasets/External_Dataset-Others/data/custom/images"
img_names =  [ os.path.join(img_folder,label_list[i].replace(".txt",".jpg")) for i in range(len(label_list))]

In [12]:
# Path to the reference slide
ref_slide_path = "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi_v2/XE07-047_1_AmyB_1/image/XE07-047_1_AmyB_1_8381x_120830y_image.png"
# Initialize the normalizer
normalizer = initialize_normalizer(ref_slide_path)

Initializing normalization with reference slide: /gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi_v2/XE07-047_1_AmyB_1/image/XE07-047_1_AmyB_1_8381x_120830y_image.png


In [13]:
def normalize_slide(target_image, normalizer):
    target_image = Vips.Image.new_from_memory(target_image.data, target_image.shape[1], target_image.shape[0], 3, "uchar")
    # Perform normalization
    normalized_image = normalizer.transform(target_image)
    return normalized_image

In [14]:
def change_to_numpy_array(input_tensor):
    input_tensor_cpu = input_tensor.squeeze(0).cpu()  # Remove batch dimension and move to CPU if necessary
    # Convert to NumPy array
    numpy_array = input_tensor_cpu.permute(1, 2, 0).numpy()  # Convert to HWC format for visualization
    # Scale back to 0-255 range if needed
    numpy_array = (numpy_array * 255).astype(np.uint8)
    return numpy_array

In [96]:
# Choose a font
myFont = ImageFont.truetype('FreeMono.ttf', 30)

# Add text
text = "Hello, World!"
text_color = (255, 255, 255)  # White
text_position = (10, 10)  # Top-left corner


In [108]:
map_class ={0:"CAA",1:"Cored"}
img_names_list_norm = []
act_label_list_norm = []
pred_label_list_norm =[ ]
for i in range(len(img_names)):
    image = Image.open(img_names[i])
    label_path = os.path.join(path, label_list[i])
    gt_boxes = np.loadtxt(label_path).reshape(-1, 5)
    gt_labels = gt_boxes[:,0]
    gt_bboxes = gt_boxes[:,1:]
    gt_bboxes_form = [ convert_bbox(box) for box in gt_bboxes ]
    crops, cropped_bboxes, cropped_labels =  zoom_and_crop(np.array(image), gt_bboxes_form,gt_labels,  zoom_factor=2, crop_size=(1024, 1024))
    for crop, cropped_bbox, cropped_label in zip( crops, cropped_bboxes, cropped_labels):
        if len(cropped_bbox)>0:
            image_inp_norm =  normalize_slide(np.array(crop), normalizer)
            input_tensor, image_float_np = prepare_input(np.array(image_inp_norm))
            masks, pred_boxes, pred_labels, pred_scores = get_outputs_nms(input_tensor, model, 0.6, 0.25)
            for j in range(len(cropped_bbox)):
                for k in range(len(pred_boxes)):
                    box1 = torch.tensor([cropped_bbox[j]], dtype=torch.float)
                    box2 = torch.tensor([pred_boxes[k]], dtype=torch.float)
                    iou = bops.box_iou(box1, box2)
                    if iou>0:
                        image_1024 = change_to_numpy_array(input_tensor)
                        bgr_img = Image.fromarray(image_1024)
                        draw = ImageDraw.Draw(bgr_img)
                        draw.rectangle(cropped_bbox[j] ,outline='red')
                        draw.text((cropped_bbox[j][0],cropped_bbox[j][1]) , "GT: "+map_class[cropped_label[j]], 'red',myFont)
                        draw.rectangle(pred_boxes[k] ,outline='green')
                        draw.text((pred_boxes[k][2],pred_boxes[k][3]), "Pred: "+pred_labels[k], 'green',myFont)
                        img_name_output = img_names[i].split("/")[-1]
                        bgr_img.save(os.path.join("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/output_text_formatted_bbox",str(j)+str(k)+"_"+"GT_"+map_class[cropped_label[j]] +"_"+"pred_"+pred_labels[k]+"_"+img_name_output))
                        img_names_list_norm.append(img_names[i].split("/")[-1])
                        act_label_list_norm.append(cropped_label[j])
                        pred_label_list_norm.append(pred_labels[k])


In [97]:
len(img_names_list)

1857

In [98]:
len(act_label_list)

1855

In [99]:
len(pred_label_list)

1855

In [103]:
output = pd.DataFrame({"act_label":act_label_list, "pred_label":pred_label_list})

In [105]:
output["act_label"] = output["act_label"].apply(lambda l: "CAA" if l==0 else "Cored")

In [109]:
output[(output["act_label"]==output["pred_label"])]

,act_label,pred_label
26,CAA,CAA
39,CAA,CAA
45,CAA,CAA
50,Cored,Cored
53,CAA,CAA
61,Cored,Cored
62,CAA,CAA
67,CAA,CAA
68,CAA,CAA
71,CAA,CAA


In [121]:
output["act_label"].value_counts()

Cored    1180
CAA       675
Name: act_label, dtype: int64

In [122]:
output["pred_label"].value_counts()

Diffuse           1841
CAA                  9
Coarse-Grained       3
Cored                2
Name: pred_label, dtype: int64

In [79]:
output_norm = pd.DataFrame({"image_name":img_names_list_norm,"act_label":act_label_list_norm, "pred_label":pred_label_list_norm})

In [80]:
output_norm["act_label"] = output_norm["act_label"].apply(lambda l: "CAA" if l==0 else "Cored")

In [81]:
output_norm

,image_name,act_label,pred_label
0,cw18-015_0_10_15.jpg,Cored,Diffuse
1,cw18-015_0_10_15.jpg,Cored,Coarse-Grained
2,cw18-015_0_10_15.jpg,Cored,Coarse-Grained
3,cw18-015_0_10_15.jpg,Cored,Coarse-Grained
4,cw17-018_0_8_27.jpg,Cored,Coarse-Grained
...,...,...,...
1411,NA4916UCDtemporalgyri _4G8_0_18_17.jpg,Cored,Diffuse
1412,cw17-022_0_19_15.jpg,CAA,CAA
1413,cw17-022_0_19_15.jpg,CAA,CAA
1414,cw17-022_0_19_15.jpg,CAA,CAA


In [82]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
confusion_matrix(output_norm["act_label"], output_norm["pred_label"])

array([[124,   9,   1, 135],
       [  0,   0,   0,   0],
       [ 46, 306, 150, 645],
       [  0,   0,   0,   0]])

In [83]:
precision_recall_fscore_support(output_norm["act_label"], output_norm["pred_label"])

/home/mahirwar/miniconda3/envs/kfold_amy_plaque1/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(array([0.72941176, 0.        , 0.99337748, 0.        ]),
 array([0.46096654, 0.        , 0.13077594, 0.        ]),
 array([0.56492027, 0.        , 0.23112481, 0.        ]),
 array([ 269,    0, 1147,    0]))

In [84]:
output_norm["pred_label1"] = output_norm["pred_label"].apply(lambda l: "Cored" if l=="Coarse-Grained" or l=="Diffuse" else l)

In [85]:
output_norm["pred_label1"].value_counts()

Cored    1246
CAA       170
Name: pred_label1, dtype: int64

In [86]:
output_norm["act_label"].value_counts()

Cored    1147
CAA       269
Name: act_label, dtype: int64

In [87]:
output_norm["pred_label1"] = output_norm["pred_label"].apply(lambda l: "Cored" if l=="Coarse-Grained" or l=="Diffuse" else l)
confusion_matrix(output_norm["act_label"], output_norm["pred_label1"],labels=["CAA", "Cored"])


array([[ 124,  145],
       [  46, 1101]])

In [88]:
precision_recall_fscore_support(output_norm["act_label"], output_norm["pred_label1"],labels=["CAA", "Cored"])

(array([0.72941176, 0.88362761]),
 array([0.46096654, 0.95989538]),
 array([0.56492027, 0.92018387]),
 array([ 269, 1147]))

In [89]:
output_norm["pred_label1"] = output_norm["pred_label"].apply(lambda l: "Cored" if l=="Coarse-Grained" else l)
confusion_matrix(output_norm["act_label"], output_norm["pred_label1"])

array([[124,  10, 135],
       [ 46, 456, 645],
       [  0,   0,   0]])

In [90]:
output_norm["pred_label1"].value_counts()

Diffuse    780
Cored      466
CAA        170
Name: pred_label1, dtype: int64

In [91]:
precision_recall_fscore_support(output_norm["act_label"], output_norm["pred_label1"])

/home/mahirwar/miniconda3/envs/kfold_amy_plaque1/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(array([0.72941176, 0.97854077, 0.        ]),
 array([0.46096654, 0.39755885, 0.        ]),
 array([0.56492027, 0.56540608, 0.        ]),
 array([ 269, 1147,    0]))

In [92]:
output_norm.to_csv("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/external_dataset_2_final_results.csv")